### First, let's bring in some data.

In [1]:
# Imports and helper functions

import accidentHandler as aH
import numpy as np
import pandas as pd
import riskModelBuilder as rMB
import seaborn as sb
from matplotlib import pyplot as plt

from sklearn import preprocessing as skpre
from sklearn import linear_model
from sklearn.cross_validation import cross_val_predict
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from sklearn import metrics

%matplotlib inline

### Accident data.

In [ ]:
accdf = pd.read_csv('/home/louisf/Documents/Insight/massdriver/data/accidents_3yr', low_memory=False)

### Next, let's load in a post-imputed data set. We then add a new column which is the number of accidents.

In [ ]:
roaddf = pd.read_csv('/home/louisf/Documents/Insight/massdriver/data/intermediateGraphDB.csv')



In [ ]:
accdf.columns

In [ ]:
accdf['numaccidents']=1

In [ ]:
numaccidents = grouped.agg(sum).numaccidents
numaccidents.index.name = 'roadsegmen'

In [ ]:
df2 = roaddf.join(numaccidents, on=road)
df2.numaccidents[pd.isnull(df2.numaccidents)] = 0

In [ ]:
df2['accidentspersegment']=df2['numaccidents']/df2['assignedle']

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.hist(df2.numaccidents/df2., bins=300)
plt.yscale('log')
plt.title('Number of accidents per road segment')
plt.xlabel('Number of accidents')
plt.ylabel('Number of road segments')
plt.savefig('numberofAccidents.png', dpi=300)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.hist(df2.accidentspersegment, bins=np.logspace(0.1, 4,150 )) #pl.hist(data, bins=np.logspace(0.1, 1.0, 50))
plt.yscale('log')
plt.xscale('log')
plt.title('Accident rate per road segment')
plt.xlabel('Accident rate')
plt.ylabel('Number of road segments')
plt.savefig('rateofAccidents.png', dpi=300)

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = plt.hist(2+df2.accidentspersegment, bins=np.logspace(.1,4,150)) #pl.hist(data, bins=np.logspace(0.1, 1.0, 50))
plt.yscale('log')
plt.xscale('log')
plt.title('Accident rate per road segment')
plt.xlabel('Accident rate + 1')
plt.ylabel('Number of road segments')
plt.savefig('rateofAccidents.png', dpi=300)

### Now it's time to start assigning categorical variables to this mess.

In [ ]:
# Bring in the requirements document.
pathtoreqs = '/home/louisf/Documents/Insight/massdriver/docs/road_inventory_params.csv'
reqs = pd.read_csv(pathtoreqs, delimiter = '^')

In [ ]:
replaceMe = reqs[(reqs['Categorical']==True) & (reqs['Disqualify']==False)]
test = roaddf.copy()

In [ ]:
for variable in replaceMe.Variable:
    dummycat = pd.get_dummies(test[variable], prefix=variable)
    test.drop(variable, inplace=True, axis=1)
    test = test.join(dummycat, how='right')

In [ ]:
dropdisq = reqs[reqs['Disqualify']==True]
for var in dropdisq['Variable']:
    try:
        test.drop(var, inplace=True, axis=1)
    except ValueError:
        print("{} not found".format(var))

###  Clear the memory here!

In [ ]:
del reqs
del roaddf
del grouped
del dropdisq

In [2]:
catDF = pd.DataFrame.from_csv('~/Documents/Insight/massdriver/data/categoricalDF.csv')

In [3]:
nparr = np.asarray(catDF)

### Predict!

In [4]:
m = rMB.Model()

In [5]:
# Split up the data 

test_size = 0.2

coltodrop = [0, 14, 640, 641]
X = np.delete(nparr, coltodrop, axis=1)
# X = X[1:20000,:]
y = nparr[:,-2]
# y = y[1:20000]
x_train, x_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size)

del nparr

In [6]:
n_estimators = [10, 50, 100, 200, 300, 400, 500]
scores = []
oob_scores = []
rf_preds = []
xvs = []
yvs = []
rfs = []
for n in n_estimators:
    score, oob_score, rf_pred, xv, yv, rf = m.rf_regression(x_train, y_train, n, .3)
    scores.append(score)
    oob_scores.append(oob_score)
    rfs.append(rf)
    print('n estimators: {} completed'.format(n))

/home/louisf/anaconda3/lib/python3.5/site-packages/sklearn/ensemble/forest.py:687: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


n estimators: 10 completed
n estimators: 50 completed


KeyboardInterrupt: 

In [8]:
oob_scores

[0.10755047001108298, 0.022903001242484344]

In [ ]:
del xvs
del yvs
del rf_preds

In [ ]:
# Perform 5 fold cross validation during training

predicted = m.rf_regression_crossval(x_train, y_train, 100, 5)

In [ ]:
metrics.r2_score(y_train, predicted)

In [ ]:
m.linear_crossval(x_train, y_train, 5)

In [ ]:
# Perform training 
score, oob_score, rf_pred, x_val, y_val, rf = m.rf_regression(x_train, y_train, 100, 0.2)

In [ ]:
score2, oob_score2, rf_pred2, x_val2, y_val2, rf2 = m.rf_regression(x_train, y_train, 200, 0.2)